## PDF Generation Workflow

Notebook for generating PDFs from resume-job matches with PII integration.

### Configuration:
- Set `SELECTION_CHOICE` for file selection (1-4)
- Configure `TREATMENT_TYPES` to process
- API settings loaded from `.env` file
- Results saved to data folder

### Workflow:
1. Load configuration and data
2. Select files to process
3. Generate PII data for each file/treatment
4. Test with single file (optional)
5. Process all files with production webhook
6. Save results and metadata to CSV

### Data Sources:
- Country mapping: `country_cluster_mapping.csv`
- PII clusters: `PII_country_cluster_mapping.csv`

### Imports

In [4]:
# Required imports
import requests
import json
import pandas as pd
import os
from datetime import datetime
import time  
import random
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
print("Test: ",os.getenv("TEST_WEBHOOK_URL"))
print("Prod: ",os.getenv("PRODUCTION_WEBHOOK_URL"))

# Excel file reading support
import openpyxl


Test:  https://adhil-is-dummy.app.n8n.cloud/webhook-test/1980a2d5-7906-462b-8d95-ae2ba0579a36
Prod:  https://adhil-is-dummy.app.n8n.cloud/webhook/1980a2d5-7906-462b-8d95-ae2ba0579a36


### Configuration

In [ ]:
# Configuration Settings
# =====================

# File Processing Configuration
SELECTION_CHOICE = "1"  # Options: "1" (all files), "2" (by number), "3" (by filename), "4" (first N files)
FILE_NUMBERS = [1]  # For option 2 - specific file numbers (1-based indexing)
FILENAMES = [ ]  # For option 3 - specific filenames
NUM_FILES_TO_PROCESS = 4  # For option 4 - number of files to process

# Treatment Configuration
TREATMENT_TYPES = ['control', 'Type_I', 'Type_II', 'Type_III']

# Reprocessing Configuration
REPROCESS_ALREADY_PROCESSED = False  # Set to True to reprocess already processed files

# Excel Integration Configuration
EXCEL_FILE_PATH = r"C:\Users\n01780947\OneDrive\Documents\ACCESS_CCSIF_Audit_Study_PDFs\Mapping_data.xlsx"
EXCEL_SHEET_NAME = "data_repo"

# API Configuration - Load from .env file only
TEST_WEBHOOK_URL = os.getenv('TEST_WEBHOOK_URL')
PRODUCTION_WEBHOOK_URL = os.getenv('PRODUCTION_WEBHOOK_URL')
WEBHOOK_USERNAME = os.getenv('WEBHOOK_USERNAME')
WEBHOOK_PASSWORD = os.getenv('WEBHOOK_PASSWORD')

# Validate that all required environment variables are set
required_env_vars = {
    'TEST_WEBHOOK_URL': TEST_WEBHOOK_URL,
    'PRODUCTION_WEBHOOK_URL': PRODUCTION_WEBHOOK_URL,
    'WEBHOOK_USERNAME': WEBHOOK_USERNAME,
    'WEBHOOK_PASSWORD': WEBHOOK_PASSWORD
}

missing_vars = [var for var, value in required_env_vars.items() if not value]
if missing_vars:
    raise ValueError(f"Missing required environment variables: {missing_vars}")

AUTHORIZATION = (WEBHOOK_USERNAME, WEBHOOK_PASSWORD)

# Template Configuration
TEMPLATE_ASSIGNMENTS = {
    'control': {'template_id': '72b77b23d48f366e', 'markdown_template': 1},
    'Type_I': {'template_id': '73677b23d4896786', 'markdown_template': 2},
    'Type_II': {'template_id': '2ee77b23de5fc78e', 'markdown_template': 2},
    'Type_III': {'template_id': 'ca277b23d48328b0', 'markdown_template': 1}
}

print("Configuration Loaded:")
print(f"- Selection Choice: {SELECTION_CHOICE}")
print(f"- Reprocess Already Processed: {REPROCESS_ALREADY_PROCESSED}")
print(f"- Treatment Types: {TREATMENT_TYPES}")
print(f"- Excel File: {EXCEL_FILE_PATH}")
print(f"- Templates Configured: {len(TEMPLATE_ASSIGNMENTS)}")
print(f"- API Configuration: ✅ Loaded from .env file")

Configuration Loaded:
- Selection Choice: 1
- Reprocess Already Processed: False
- Treatment Types: ['control', 'Type_I', 'Type_II', 'Type_III']
- Excel File: C:\Users\n01780947\OneDrive\Documents\ACCESS_CCSIF_Audit_Study_PDFs\Mapping_data.xlsx
- Templates Configured: 4
- API Configuration: ✅ Loaded from .env file


### Data Loading

In [7]:
# Load Job Matches Data
# ====================

# Load the job matches data
file_path = "LC_batch1_resume_matches.csv"
job_matches_df = pd.read_csv(file_path)

# Rename columns to match our endpoint requirements
job_matches_df.rename(columns={'tile': 'job_title'}, inplace=True)

print(f"✅ Loaded {len(job_matches_df)} job matches")
print(f"📋 Columns: {list(job_matches_df.columns)}")

# Display first few rows
print(f"\n📊 Sample data:")
job_matches_df.head()

✅ Loaded 16 job matches
📋 Columns: ['_id', 'job_posting_id', 'job_link', 'title', 'company', 'location', 'job_description', 'file_id', 'key_metrics.basics.likely_home_country', 'match_score', 'link_status', 'unsupported_input_fields', 'unsupported_input_field_labels[0]', 'unsupported_input_field_labels[1]']

📊 Sample data:


,_id,job_posting_id,job_link,title,company,location,job_description,file_id,key_metrics.basics.likely_home_country,match_score,link_status,unsupported_input_fields,unsupported_input_field_labels[0],unsupported_input_field_labels[1]
0,6914e28fd4563fbd0dbc00cc,68f540b7345bb9f51a02633d,https://job-boards.greenhouse.io/gomotive/jobs...,"Associate Risk Manager, Canada",Motive,"Toronto, ON","# Associate Risk Manager, Canada\n\nMotive emp...",LC Resume 03.pdf,Nigeria,95,active,False,NaN,NaN
1,6914e585d4563fbd0dbc0120,6912444ba2346b40b4c7b615,https://job-boards.greenhouse.io/roofr/jobs/46...,"Program Manager, Partnerships",Roofr,"Toronto, ON","# Program Manager, Partnerships\n\nWe’re looki...",LC Resume 15.pdf,India,95,active,False,NaN,NaN
2,6914e6a3d4563fbd0dbc0143,6912444da2346b40b4c7b64d,https://job-boards.greenhouse.io/capintel/jobs...,Product Implementation Manager,CapIntel,"Toronto, ON",# Product Implementation Manager\n\nAbout the ...,LC Resume 16.pdf,India,85,active,False,Which city do you reside in?*,NaN
3,6914e6d3d4563fbd0dbc014a,6912444da2346b40b4c7b660,https://boards.greenhouse.io/touchbistro/jobs/...,Product Manager,TouchBistro,"Toronto, ON",# Product Manager\n\n* Turn insights into ac...,LC Resume 16.pdf,India,92,active,False,NaN,NaN
4,6914ea30d4563fbd0dbc01a6,691244a7a2346b40b4c7b703,https://job-boards.greenhouse.io/metalab/jobs/...,Senior Product Manager (Contract),Metalab,"Vancouver, BC",# Senior Product Manager (Contract)\n\n**Your ...,LC Resume 16.pdf,India,95,active,False,Location (City)*,NaN


### File Selection Logic

In [8]:
# Updated File Selection Logic with Treatment-Level Filtering
# ==========================================================

# Configuration for Excel file location
EXCEL_FILE_PATH = r"C:\Users\n01780947\OneDrive\Documents\ACCESS_CCSIF_Audit_Study_PDFs\Mapping_data.xlsx"
EXCEL_SHEET_NAME = "data_repo"

# Load Job Matches Data
# ====================

# Load the job matches data

job_matches_df = pd.read_csv(file_path)

# Rename columns to match our endpoint requirements
job_matches_df.rename(columns={'tile': 'job_title'}, inplace=True)

print(f"✅ Loaded {len(job_matches_df)} job matches")
print(f"📋 Columns: {list(job_matches_df.columns)}")

# Load existing results from Excel file
existing_results = []
processed_combinations = set()  # Store (job_id, treatment_type) combinations

try:
    # Read the Excel file from OneDrive location
    excel_df = pd.read_excel(EXCEL_FILE_PATH, sheet_name=EXCEL_SHEET_NAME)
    print(f"✅ Loaded {len(excel_df)} existing results from Excel file")
    print(f"📋 Excel columns: {list(excel_df.columns)}")
    
    # Get processed job-treatment combinations
    if not REPROCESS_ALREADY_PROCESSED:
        # Check for both 'id' and 'treatment_type' columns
        if 'match_id' in excel_df.columns and 'treatment_type' in excel_df.columns:
            # Create combinations of (job_id, treatment_type) that have been processed
            for _, row in excel_df.iterrows():
                job_id = str(row['match_id']) if pd.notna(row['match_id']) else None
                treatment = str(row['treatment_type']) if pd.notna(row['treatment_type']) else None
                if job_id and treatment:
                    processed_combinations.add((job_id, treatment))
            
            print(f"🔍 Found {len(processed_combinations)} already processed job-treatment combinations")
            
            # Analyze completeness
            job_ids_in_excel = set(excel_df['match_id'].dropna().astype(str))
            print(f"📊 Job IDs in Excel: {len(job_ids_in_excel)}")
            
            # Check which jobs have incomplete treatments
            incomplete_jobs = []
            for job_id in job_ids_in_excel:
                processed_treatments = [t for j, t in processed_combinations if j == job_id]
                missing_treatments = [t for t in TREATMENT_TYPES if t not in processed_treatments]
                if missing_treatments:
                    incomplete_jobs.append((job_id, missing_treatments))
            
            if incomplete_jobs:
                print(f"⚠️  Found {len(incomplete_jobs)} jobs with incomplete treatments:")
                for job_id, missing in incomplete_jobs[:5]:  # Show first 5
                    print(f"   Job {job_id}: Missing {missing}")
                if len(incomplete_jobs) > 5:
                    print(f"   ... and {len(incomplete_jobs) - 5} more")
        else:
            print("⚠️  Warning: Required columns ('id' or 'treatment_type') not found in Excel file")
    
except FileNotFoundError:
    print(f"⚠️  Excel file not found at: {EXCEL_FILE_PATH}")
    print("   Will process all files as if none have been processed")
except Exception as e:
    print(f"⚠️  Error reading Excel file: {str(e)}")
    print("   Will process all files as if none have been processed")

# Filter job matches based on processing status
if not REPROCESS_ALREADY_PROCESSED and processed_combinations:
    print(f"\n🔍 Filtering job matches based on processing status...")
    
    # Create a list to store job matches that need processing
    jobs_to_process = []
    
    for _, job_row in job_matches_df.iterrows():
        job_id = str(job_row['_id'])
        processed_treatments = [t for j, t in processed_combinations if j == job_id]
        missing_treatments = [t for t in TREATMENT_TYPES if t not in processed_treatments]
        
        if missing_treatments:
            # This job needs processing for missing treatments
            jobs_to_process.append({
                'job_row': job_row,
                'missing_treatments': missing_treatments,
                'processed_treatments': processed_treatments
            })
    
    print(f"📊 Jobs needing processing: {len(jobs_to_process)}")
    
    # Create filtered dataframe with only jobs that need processing
    job_ids_to_process = [job['job_row']['_id'] for job in jobs_to_process]
    all_files_df = job_matches_df[job_matches_df['_id'].isin(job_ids_to_process)]
    
    print(f"📊 Job matches to process: {len(all_files_df)}")
    
    # Show summary of what needs to be processed
    total_missing_treatments = sum(len(job['missing_treatments']) for job in jobs_to_process)
    print(f"🔄 Total missing treatments to process: {total_missing_treatments}")
    
else:
    all_files_df = job_matches_df.copy()
    print(f"📊 Processing all {len(all_files_df)} job matches")

# Display first 10 files with their details
print(f"\n📋 Sample files to process (first 10):")
print("-" * 60)
for idx, row in all_files_df.head(10).iterrows():
    print(f"{idx+1:2d}. {row['file_id']}")
    if 'key_metrics.basics.likely_home_country' in row and pd.notna(row['key_metrics.basics.likely_home_country']):
        print(f"    Country: {row['key_metrics.basics.likely_home_country']}")
    if 'location' in row and pd.notna(row['location']):
        print(f"    Location: {row['location']}")
    print(f"    Job Title: {row.get('title', 'N/A')}")
    
    # Show treatment status if we have processing info
    if not REPROCESS_ALREADY_PROCESSED and processed_combinations:
        job_id = str(row['_id'])
        processed_treatments = [t for j, t in processed_combinations if j == job_id]
        missing_treatments = [t for t in TREATMENT_TYPES if t not in processed_treatments]
        print(f"    Processed: {processed_treatments}")
        print(f"    Missing: {missing_treatments}")
    print()

if len(all_files_df) > 10:
    print(f"... and {len(all_files_df) - 10} more files")

# File selection logic
print(f"🎯 File Selection Options:")
print("1. Process all files")
print("2. Process specific files by number")
print("3. Process specific files by filename") 
print("4. Process first N files")
print(f"\nCurrent selection: Option {SELECTION_CHOICE}")
print(f"Reprocess already processed: {REPROCESS_ALREADY_PROCESSED}")

# Apply file selection logic
if SELECTION_CHOICE == "1":
    # Process all files
    files_to_process = all_files_df
    print(f"\n✅ Processing ALL {len(files_to_process)} job matches")
    
elif SELECTION_CHOICE == "2":
    # Process specific files by number
    print(f"\n✅ Processing specific files by number: {FILE_NUMBERS}")
    try:
        valid_indices = [i - 1 for i in FILE_NUMBERS if 1 <= i <= len(all_files_df)]
        if valid_indices:
            files_to_process = all_files_df.iloc[valid_indices]
            print(f"Selected {len(files_to_process)} job matches:")
            for idx, row in files_to_process.iterrows():
                print(f"  - {row['file_id']} (Job: {row.get('job_posting_id', 'N/A')})")
        else:
            print("⚠️  No valid file numbers provided. Processing first file.")
            files_to_process = all_files_df.head(1)
    except Exception as e:
        print(f"❌ Error: {e}. Processing first file.")
        files_to_process = all_files_df.head(1)
        
elif SELECTION_CHOICE == "3":
    # Process specific files by filename
    print(f"\n✅ Processing specific files by filename: {FILENAMES}")
    try:
        files_to_process = all_files_df[all_files_df['file_id'].isin(FILENAMES)]
        if len(files_to_process) > 0:
            print(f"Selected {len(files_to_process)} job matches:")
            for idx, row in files_to_process.iterrows():
                print(f"  - {row['file_id']} (Job: {row.get('job_posting_id', 'N/A')})")
        else:
            print("⚠️  No matching filenames found. Processing first file.")
            files_to_process = all_files_df.head(1)
    except Exception as e:
        print(f"❌ Error: {e}. Processing first file.")
        files_to_process = all_files_df.head(1)
        
elif SELECTION_CHOICE == "4":
    # Process first N files
    files_to_process = all_files_df.head(NUM_FILES_TO_PROCESS)
    print(f"\n✅ Processing first {NUM_FILES_TO_PROCESS} job matches:")
    for idx, row in files_to_process.iterrows():
        print(f"  - {row['file_id']} (Job: {row.get('job_posting_id', 'N/A')})")
        
else:
    # Default to first file if invalid input
    print("⚠️  Invalid choice. Processing first file.")
    files_to_process = all_files_df.head(1)

print(f"\n🎯 Final selection: {len(files_to_process)} job matches to process")

# Calculate total operations (only missing treatments)
if not REPROCESS_ALREADY_PROCESSED and processed_combinations:
    total_operations = 0
    for _, job_row in files_to_process.iterrows():
        job_id = str(job_row['_id'])
        processed_treatments = [t for j, t in processed_combinations if j == job_id]
        missing_treatments = [t for t in TREATMENT_TYPES if t not in processed_treatments]
        total_operations += len(missing_treatments)
    print(f"🔄 Total operations to process (missing treatments only): {total_operations}")
else:
    total_operations = len(files_to_process) * len(TREATMENT_TYPES)
    print(f"🔄 Total operations to process: {total_operations}")

✅ Loaded 16 job matches
📋 Columns: ['_id', 'job_posting_id', 'job_link', 'title', 'company', 'location', 'job_description', 'file_id', 'key_metrics.basics.likely_home_country', 'match_score', 'link_status', 'unsupported_input_fields', 'unsupported_input_field_labels[0]', 'unsupported_input_field_labels[1]']
✅ Loaded 1471 existing results from Excel file
📋 Excel columns: ['u_id', 'match_id', 'base _resume_file_id', 'treatment_type', 'PDF_google_drive_link', 'resume_filename', 'full_name', 'email', 'phone', 'first_name', 'last_name', 'gender', 'geographic_cluster', 'likely_home_country', 'company', 'job_title', 'job_url_direct', 'job_description', 'resume_final_md', 'match_score', 'Job_location', 'timestamp', 'workflow_mode', 'validation_score', 'validation_json', 'job_id', 'PDF_download_link', 'id']
🔍 Found 1464 already processed job-treatment combinations
📊 Job IDs in Excel: 366

🔍 Filtering job matches based on processing status...
📊 Jobs needing processing: 16
📊 Job matches to proces

### PII Data Loading and Functions

In [9]:
# PII Data Loading and Functions
# =============================

# Load the country to geographic cluster mapping
country_cluster_df = pd.read_csv("country_cluster_mapping.csv")
print(f"✅ Loaded country mapping: {len(country_cluster_df)} countries mapped to {country_cluster_df['Geographic_Cluster'].nunique()} clusters")

# Load the PII clusters data
pii_clusters_df = pd.read_csv("PII_country_cluster_mapping.csv")
print(f"✅ Loaded PII clusters: {len(pii_clusters_df)} cluster-treatment combinations")

# Create lookup dictionaries for efficient access
country_to_cluster = dict(zip(country_cluster_df['Country'], country_cluster_df['Geographic_Cluster']))

# Create a nested dictionary for PII lookup: {cluster: [pii_records]}
pii_lookup = {}
for _, row in pii_clusters_df.iterrows():
    cluster = row['Geographic_Cluster']
    
    if cluster not in pii_lookup:
        pii_lookup[cluster] = []
    
    # Parse the name pools into lists
    male_names = [name.strip() for name in row['Male_First_Name_Pool'].split(',')]
    female_names = [name.strip() for name in row['Female_First_Name_Pool'].split(',')]
    
    pii_record = {
        'pii_id': row['PII_Identifier_ID'],
        'treatment_type': row['Treatment Type'],
        'last_name': row['Last_Name'],
        'email': row['Assigned_Email'],
        'phone': row['Assigned_Phone_Number'],
        'male_names': male_names,
        'female_names': female_names
    }
    
    pii_lookup[cluster].append(pii_record)

print(f"\n📋 PII lookup structure created:")
for cluster in pii_lookup:
    print(f"  {cluster}: {len(pii_lookup[cluster])} PII records")

# Function to get cluster for a country with fallback to Anglosphere
def get_cluster_for_country(country):
    """
    Get geographic cluster for a country, with fallback to Anglosphere if not found.
    
    Args:
        country (str): Country name
    
    Returns:
        str: Geographic cluster name
    """
    if pd.isna(country) or country == 'Unknown' or country == '':
        print(f"⚠️  Country '{country}' is missing/unknown, using Anglosphere fallback")
        return 'Anglosphere'
    
    if country in country_to_cluster:
        return country_to_cluster[country]
    else:
        print(f"⚠️  Country '{country}' not found in mapping, using Anglosphere fallback")
        return 'Anglosphere'

# Helper function to generate PII from a selected record
def generate_pii_from_record(pii_record):
    """
    Generate PII data from a selected PII record.
    
    Args:
        pii_record (dict): Selected PII record
    
    Returns:
        dict: Complete PII data with full_name, email, phone, etc.
    """
    # Randomly select gender (50/50 chance)
    is_male = random.choice([True, False])
    
    # Select random name from appropriate pool
    if is_male:
        first_name = random.choice(pii_record['male_names'])
    else:
        first_name = random.choice(pii_record['female_names'])
    
    # Construct full name
    full_name = f"{first_name} {pii_record['last_name']}"
    
    return {
        'full_name': full_name,
        'email': pii_record['email'],
        'phone': pii_record['phone'],
        'first_name': first_name,
        'last_name': pii_record['last_name'],
        'gender': 'Male' if is_male else 'Female',
        'pii_id': pii_record['pii_id'],
        'pii_treatment_type': pii_record['treatment_type']
    }

# Test the functions with a sample
print(f"\n🧪 Testing PII generation:")
test_country = "India"
cluster = get_cluster_for_country(test_country)
print(f"  Country: {test_country} → Cluster: {cluster}")
print(f"  Available PII records: {len(pii_lookup[cluster])}")
for i, pii in enumerate(pii_lookup[cluster]):
    print(f"    {i+1}. {pii['pii_id']} ({pii['treatment_type']}) - {pii['last_name']}")

✅ Loaded country mapping: 63 countries mapped to 7 clusters
✅ Loaded PII clusters: 28 cluster-treatment combinations

📋 PII lookup structure created:
  South Asia: 4 PII records
  Sub-Saharan Africa: 4 PII records
  Middle East & North Africa: 4 PII records
  East & Southeast Asia: 4 PII records
  Eastern Europe: 4 PII records
  Latin America: 4 PII records
  Anglosphere: 4 PII records

🧪 Testing PII generation:
  Country: India → Cluster: South Asia
  Available PII records: 4
    1. SA-01 (control) - Patel
    2. SA-02 (Type_I) - Singh
    3. SA-03 (Type_II) - Kumar
    4. SA-04 (Type_III) - Khan


### Apply PII Data to Selected Files

In [10]:
# Apply PII Data to Selected Files - All 4 PIIs per Job Match
# ==========================================================

# Apply PII selection to the files we want to process
print("🔄 Generating PII data for selected files...")
print("-" * 60)

files_with_pii = []

for idx, row in files_to_process.iterrows():
    file_id = row['file_id']
    country = row.get('key_metrics.basics.likely_home_country', 'Unknown')
    
    print(f"\n📄 File: {file_id}")
    print(f"   Country: {country}")
    
    # Get cluster for this country
    cluster = get_cluster_for_country(country)
    
    # Get all 4 PII records for this cluster
    cluster_pii_records = pii_lookup.get(cluster, [])
    
    if len(cluster_pii_records) != 4:
        print(f"   ⚠️  Warning: Expected 4 PII records for cluster '{cluster}', found {len(cluster_pii_records)}")
    
    if cluster_pii_records:
        # Shuffle the 4 PII records to randomize assignment
        shuffled_pii_records = cluster_pii_records.copy()
        random.shuffle(shuffled_pii_records)
        
        print(f"   ✅ Using all 4 PII records from cluster '{cluster}':")
        for i, pii_record in enumerate(shuffled_pii_records):
            print(f"     {i+1}. {pii_record['pii_id']} ({pii_record['treatment_type']}) - {pii_record['last_name']}")
        
        # Generate PII data for each treatment using the shuffled records
        treatment_pii_data = {}
        for i, treatment_type in enumerate(TREATMENT_TYPES):
            pii_record = shuffled_pii_records[i]
            treatment_pii_data[treatment_type] = generate_pii_from_record(pii_record)
            
            print(f"   {treatment_type}: {treatment_pii_data[treatment_type]['full_name']} ({treatment_pii_data[treatment_type]['gender']})")
        
        # Store the file data with PII
        file_data = {
            'file_row': row,
            'pii_data': treatment_pii_data,
            'pii_ids': [pii['pii_id'] for pii in shuffled_pii_records]
        }
        files_with_pii.append(file_data)
    else:
        print(f"   ❌ No PII records found for cluster '{cluster}'")
        continue

print(f"\n✅ PII generation completed for {len(files_with_pii)} files")
print(f"🔄 Total operations to process: {len(files_with_pii)} files × {len(TREATMENT_TYPES)} treatments = {len(files_with_pii) * len(TREATMENT_TYPES)} operations")

🔄 Generating PII data for selected files...
------------------------------------------------------------

📄 File: LC Resume 03.pdf
   Country: Nigeria
   ✅ Using all 4 PII records from cluster 'Sub-Saharan Africa':
     1. SSA-04 (Type_III) - Mensah
     2. SSA-02 (Type_I) - Okoro
     3. SSA-01 (control) - Adebayo
     4. SSA-03 (Type_II) - Traore
   control: Paul Mensah (Male)
   Type_I: Aissatou Okoro (Female)
   Type_II: Arthur Adebayo (Male)
   Type_III: Léo Traore (Male)

📄 File: LC Resume 15.pdf
   Country: India
   ✅ Using all 4 PII records from cluster 'South Asia':
     1. SA-04 (Type_III) - Khan
     2. SA-02 (Type_I) - Singh
     3. SA-03 (Type_II) - Kumar
     4. SA-01 (control) - Patel
   control: Arya Khan (Female)
   Type_I: Harish Singh (Male)
   Type_II: Aditya Kumar (Male)
   Type_III: Kunal Patel (Male)

📄 File: LC Resume 16.pdf
   Country: India
   ✅ Using all 4 PII records from cluster 'South Asia':
     1. SA-01 (control) - Patel
     2. SA-04 (Type_III) - Khan
 

### Webhook Request Body Creation Function

In [11]:
# Request Body Creation Function
# =============================

# Enhanced webhook request function that includes PII data
def create_enhanced_request_body(file_row, treatment_type, pii_data):
    """
    Create an enhanced request body with PII data.
    
    Args:
        file_row: Row from the job matches dataframe
        treatment_type: Treatment type being processed
        pii_data: PII data for this treatment type
    
    Returns:
        dict: Enhanced request body
    """
    # Handle location logic: Convert "Remote" to "Toronto, ON"
    location = file_row.get('location', 'Toronto, ON')
    if pd.isna(location) or location == 'Remote':
        location = 'Toronto, ON'
    
    request_body = {
        'id': file_row['_id'],
        'file_id': file_row['file_id'],
        'treatment_type': treatment_type,
        'name': pii_data['full_name'],
        'email': pii_data['email'],
        'phone': pii_data['phone'],
        'location': location,
        'first_name': pii_data['first_name'],
        'last_name': pii_data['last_name'],
        'gender': pii_data['gender'],
        'country': file_row.get('key_metrics.basics.likely_home_country', 'Unknown'),
        'geographic_cluster': country_to_cluster.get(file_row.get('key_metrics.basics.likely_home_country', ''), 'Unknown'),
        # Add job-related fields
        'company': file_row.get('company',''),
        'job_description': file_row.get('job_description', ''),
        'job_title': file_row.get('title', ''),
        'job_posting_id': file_row.get('job_posting_id', ''),
        'match_score': file_row.get('match_score', ''),
        'job_link': file_row.get('job_link', '')
    }
    
    return request_body

# Test the request body creation
print("🧪 Testing request body creation:")
print("-" * 60)

if files_with_pii:
    test_file = files_with_pii[0]
    test_treatment = TREATMENT_TYPES[0]
    test_pii = test_file['pii_data'][test_treatment]
    
    test_request = create_enhanced_request_body(
        test_file['file_row'], 
        test_treatment, 
        test_pii
    )
    
    print(f"�� Sample request body for {test_file['file_row']['file_id']} - {test_treatment}:")
    for key, value in test_request.items():
        if key == 'job_description':
            # Truncate job description for display
            display_value = value[:100] + "..." if len(str(value)) > 100 else value
            print(f"   {key}: {display_value}")
        else:
            print(f"   {key}: {value}")
else:
    print("❌ No files with PII data found. Please run the previous cell first.")

🧪 Testing request body creation:
------------------------------------------------------------
�� Sample request body for LC Resume 03.pdf - control:
   id: 6914e28fd4563fbd0dbc00cc
   file_id: LC Resume 03.pdf
   treatment_type: control
   name: Paul Mensah
   email: mensah87452@gmail.com
   phone: +1 (437) 837-0433	
   location: Toronto, ON
   first_name: Paul
   last_name: Mensah
   gender: Male
   country: Nigeria
   geographic_cluster: Sub-Saharan Africa
   company: Motive
   job_description: # Associate Risk Manager, Canada\n\nMotive empowers the people who run physical operations with tool...
   job_title: Associate Risk Manager, Canada
   job_posting_id: 68f540b7345bb9f51a02633d
   match_score: 95
   job_link: https://job-boards.greenhouse.io/gomotive/jobs/8202172002?gh_src=my.greenhouse.search


### Production File Processing

In [12]:
# Production File Processing - Simplified Error Handling with Async Support
# ======================================================

print("=== PRODUCTION PROCESSING ===")
print(f"Production webhook: {PRODUCTION_WEBHOOK_URL}")
print(f"Processing {len(files_with_pii)} files with {len(TREATMENT_TYPES)} treatments each")

# Initialize tracking variables
total_operations = len(files_with_pii) * len(TREATMENT_TYPES)
current_operation = 0
consecutive_errors = 0
max_consecutive_errors = 2  # Stop after 2 consecutive errors
success_count = 0
processing_stopped = False

print(f"\n🚀 Starting processing...")
print("=" * 60)

# Process files
for file_idx, file_data in enumerate(files_with_pii, 1):
    if processing_stopped:
        print(f"\n🛑 Processing already stopped due to consecutive errors")
        break
        
    # Extract file_row and pii_data from the correct structure
    file_row = file_data['file_row']
    file_id = file_row['file_id']
    pii_data = file_data['pii_data']
    
    print(f"\n📄 Processing file {file_idx}/{len(files_with_pii)}: {file_id}")
    print("-" * 40)
    
    # Process treatments for this file
    for treatment_idx, treatment_type in enumerate(TREATMENT_TYPES, 1):
        if processing_stopped:
            print(f"  🛑 Stopping treatment processing due to consecutive errors")
            break
            
        current_operation += 1
        treatment_pii = pii_data[treatment_type]
        
        print(f"  🔬 Treatment {treatment_idx}/{len(TREATMENT_TYPES)}: {treatment_type}")
        print(f"     👤 Using PII: {treatment_pii['full_name']} ({treatment_pii['email']})")
        
        # Get template configuration for this treatment type
        template_config = TEMPLATE_ASSIGNMENTS[treatment_type]
        print(f"     🎨 Template ID: {template_config['template_id']}")
        print(f"     📝 Markdown Template: {template_config['markdown_template']}")
        
        # Create enhanced request body
        request_body = create_enhanced_request_body(file_row, treatment_type, treatment_pii)
        request_body['template_id'] = template_config['template_id']
        request_body['markdown_template'] = template_config['markdown_template']
        
        print(f"     📤 Sending request...")
        
        try:
            # Send request to n8n webhook (async mode - accepts immediate ack)
            response = requests.post(
                PRODUCTION_WEBHOOK_URL,
                json=request_body,
                auth=AUTHORIZATION,
                timeout=10  # Short timeout for immediate acceptance response
            )
            
            # Check for HTTP errors
            if response.status_code != 200:
                consecutive_errors += 1
                print(f"     ❌ HTTP Error: {response.status_code}")
                print(f"     🔢 Consecutive errors: {consecutive_errors}/{max_consecutive_errors}")
                
                if consecutive_errors >= max_consecutive_errors:
                    print(f"\n🛑 STOPPING EXECUTION: {consecutive_errors} consecutive errors detected!")
                    processing_stopped = True
                    break
                continue
            
            # Parse response JSON
            try:
                response_data = response.json()
            except json.JSONDecodeError:
                consecutive_errors += 1
                print(f"     ❌ JSON Parse Error")
                print(f"     🔢 Consecutive errors: {consecutive_errors}/{max_consecutive_errors}")
                
                if consecutive_errors >= max_consecutive_errors:
                    print(f"\n🛑 STOPPING EXECUTION: {consecutive_errors} consecutive errors detected!")
                    processing_stopped = True
                    break
                continue
            
            # Check for immediate acceptance (async processing mode)
            status = response_data.get('status', '').lower()
            
            if status == 'accepted':
                # Success - request accepted, processing will continue async
                consecutive_errors = 0
                success_count += 1
                
                print(f"     ✅ Request accepted (processing async)")
                print(f"        🆔 ID: {response_data.get('id', 'N/A')}")
                print(f"        🔬 Treatment: {response_data.get('treatment_type', 'N/A')}")
                print(f"        📝 Message: {response_data.get('message', 'N/A')}")
                
                # Progress tracking
                progress_percent = (current_operation / total_operations) * 100
                print(f"     📊 Progress: {current_operation}/{total_operations} ({progress_percent:.1f}%)")
                
                # Wait between requests to allow n8n workflow to complete processing (~2 minutes)
                if current_operation < total_operations and not processing_stopped:
                    print(f"     ⏳ Waiting 130 seconds before next request (allowing ~2 min processing time)...")
                    time.sleep(130)  # Wait 130 seconds (2 min 10 sec) for processing to complete
                
                continue  # Move to next request
            
            # Check for n8n workflow errors in response (fallback)
            if 'error' in status or 'failed' in status or 'exception' in status:
                consecutive_errors += 1
                print(f"     ❌ n8n Workflow Error: {status}")
                print(f"     🔢 Consecutive errors: {consecutive_errors}/{max_consecutive_errors}")
                
                if consecutive_errors >= max_consecutive_errors:
                    print(f"\n🛑 STOPPING EXECUTION: {consecutive_errors} consecutive errors detected!")
                    processing_stopped = True
                    break
                continue
            
            # Fallback: Check for old-style response with PDF link (for backward compatibility)
            google_drive_link = response_data.get('Google_Drive_Link', '')
            file_id_response = response_data.get('id', '')
            file_name = response_data.get('file_name', '')
            timestamp = response_data.get('timestamp', '')
            
            if google_drive_link and file_id_response and file_name:
                # Old-style synchronous response (backward compatibility)
                consecutive_errors = 0
                success_count += 1
                
                print(f"     ✅ Success: PDF generated (synchronous mode)")
                print(f"        🔗 Google Drive Link: {google_drive_link}")
                print(f"        📄 Filename: {file_name}")
                print(f"        🆔 ID: {file_id_response}")
                print(f"        ⏰ Timestamp: {timestamp}")
                
                # Progress tracking
                progress_percent = (current_operation / total_operations) * 100
                print(f"     📊 Progress: {current_operation}/{total_operations} ({progress_percent:.1f}%)")
                
                # Wait between requests to allow n8n workflow to complete processing (~2 minutes)
                if current_operation < total_operations and not processing_stopped:
                    print(f"     ⏳ Waiting 130 seconds before next request (allowing ~2 min processing time)...")
                    time.sleep(130)  # Wait 130 seconds (2 min 10 sec) for processing to complete
                
                continue
            
            # If we get here, response is neither "accepted" nor old-style success
            # Unknown response format
            consecutive_errors += 1
            print(f"     ⚠️  Unknown response format")
            print(f"     Response: {json.dumps(response_data, indent=2)[:200]}...")
            print(f"     🔢 Consecutive errors: {consecutive_errors}/{max_consecutive_errors}")
            
            if consecutive_errors >= max_consecutive_errors:
                print(f"\n🛑 STOPPING EXECUTION: {consecutive_errors} consecutive errors detected!")
                processing_stopped = True
                break
            continue
            
        except requests.exceptions.Timeout:
            consecutive_errors += 1
            print(f"     ❌ Request Timeout (webhook didn't respond in 10s)")
            print(f"     🔢 Consecutive errors: {consecutive_errors}/{max_consecutive_errors}")
            
            if consecutive_errors >= max_consecutive_errors:
                print(f"\n🛑 STOPPING EXECUTION: {consecutive_errors} consecutive errors detected!")
                processing_stopped = True
                break
            continue
            
        except Exception as e:
            consecutive_errors += 1
            print(f"     ❌ Exception: {str(e)}")
            print(f"     🔢 Consecutive errors: {consecutive_errors}/{max_consecutive_errors}")
            
            if consecutive_errors >= max_consecutive_errors:
                print(f"\n🛑 STOPPING EXECUTION: {consecutive_errors} consecutive errors detected!")
                processing_stopped = True
                break
            continue
    
    # Check if we should stop due to consecutive errors (after processing all treatments for a file)
    if processing_stopped:
        print(f"\n🛑 STOPPING EXECUTION: {consecutive_errors} consecutive errors detected!")
        print(f"   Processing stopped at file {file_idx}/{len(files_with_pii)}")
        break

# Final summary
print(f"\n📊 Processing Summary:")
print(f"  ✅ Successful: {success_count}")
print(f"  📊 Success Rate: {(success_count/current_operation*100):.1f}%" if current_operation > 0 else "N/A")

if processing_stopped:
    print(f"\n🛑 EXECUTION STOPPED EARLY DUE TO ERRORS!")
    print(f"   Consecutive errors: {consecutive_errors}")
    print(f"   Check n8n logs for detailed error information.")
    print(f"   Fix the issues and restart the script to continue processing.")
else:
    print(f"\n🎉 Processing workflow completed successfully!")

print(f"🔄 Total operations processed: {current_operation}")

=== PRODUCTION PROCESSING ===
Production webhook: https://adhil-is-dummy.app.n8n.cloud/webhook/1980a2d5-7906-462b-8d95-ae2ba0579a36
Processing 16 files with 4 treatments each

🚀 Starting processing...

📄 Processing file 1/16: LC Resume 03.pdf
----------------------------------------
  🔬 Treatment 1/4: control
     👤 Using PII: Paul Mensah (mensah87452@gmail.com)
     🎨 Template ID: 72b77b23d48f366e
     📝 Markdown Template: 1
     📤 Sending request...
     ✅ Request accepted (processing async)
        🆔 ID: {{ $('Webhook').item.json.body.id }}
        🔬 Treatment: {{ $('Webhook').item.json.body.treatment_type }}
        📝 Message: Processing started - resume will be processed asynchronously
     📊 Progress: 1/64 (1.6%)
     ⏳ Waiting 130 seconds before next request (allowing ~2 min processing time)...
  🔬 Treatment 2/4: Type_I
     👤 Using PII: Aissatou Okoro (okoro0293x@gmail.com)
     🎨 Template ID: 73677b23d4896786
     📝 Markdown Template: 2
     📤 Sending request...
     ✅ Request

### Excel Data Analysis and Processing Report

In [102]:
### Excel Data Analysis and Processing Report (Current Batch Only)
# ==============================================================

print("=== EXCEL DATA ANALYSIS AND PROCESSING REPORT ===")
print("=" * 60)

# Configuration - Define everything needed for analysis
EXCEL_SHEET_NAME = "data_repo"
TREATMENT_TYPES = ['control', 'Type_I', 'Type_II', 'Type_III']
CSV_FILE_PATH = file_path

# Try multiple paths for the Excel file
excel_paths = [
    r"C:\Users\n01780947\OneDrive\Documents\ACCESS_CCSIF_Audit_Study_PDFs\Mapping_data.xlsx",
    r"C:\Users\n01780947\OneDrive - Humber Polytechnic\Documents\ACCESS_CCSIF_Audit_Study_PDFs\Mapping_data.xlsx",
    "Mapping_data.xlsx",  # Local copy
    "data/Mapping_data.xlsx"  # In data folder
]

excel_df = None
excel_path_used = None

for excel_path in excel_paths:
    try:
        excel_df = pd.read_excel(excel_path, sheet_name=EXCEL_SHEET_NAME)
        excel_path_used = excel_path
        print(f"✅ Successfully loaded Excel file from: {excel_path}")
        break
    except (FileNotFoundError, PermissionError) as e:
        print(f"⚠️  Could not access: {excel_path} - {str(e)}")
        continue
    except Exception as e:
        print(f"⚠️  Error reading: {excel_path} - {str(e)}")
        continue

# Load CSV data for comparison
try:
    job_matches_df = pd.read_csv(CSV_FILE_PATH)
    print(f"✅ Successfully loaded CSV file: {CSV_FILE_PATH}")
except Exception as e:
    print(f"❌ Could not load CSV file: {str(e)}")
    job_matches_df = None

if excel_df is None:
    print("❌ Could not access Excel file from any location")
    print("   Please ensure the file is accessible or copy it locally")
elif job_matches_df is None:
    print("❌ Could not load CSV file for comparison")
else:
    print(f"\n📊 EXCEL FILE OVERVIEW")
    print("-" * 40)
    print(f"📁 File Path: {excel_path_used}")
    print(f"📋 Total Records in Excel: {len(excel_df)}")
    print(f"📋 Total Records in Current Batch: {len(job_matches_df)}")
    print(f"📋 Columns: {list(excel_df.columns)}")
    
    # Processing completeness analysis - ONLY for current batch
    print(f"\n📈 PROCESSING COMPLETENESS ANALYSIS (Current Batch Only)")
    print("-" * 40)
    
    if 'match_id' in excel_df.columns and 'treatment_type' in excel_df.columns:
        # Get all unique job IDs from current batch CSV
        current_batch_job_ids = set(job_matches_df['_id'].astype(str))
        
        # Filter Excel data to ONLY include current batch jobs
        current_batch_excel = excel_df[excel_df['match_id'].isin(current_batch_job_ids)]
        
        print(f"   Current Batch Job IDs: {len(current_batch_job_ids)}")
        print(f"   Current Batch Records in Excel: {len(current_batch_excel)}")
        
        # Get unique job IDs from filtered Excel data
        excel_job_ids = set(current_batch_excel['match_id'].dropna().astype(str))
        
        print(f"   Jobs processed in Excel: {len(excel_job_ids)}")
        
        # Find jobs that are in current batch but not in Excel
        missing_from_excel = current_batch_job_ids - excel_job_ids
        print(f"   Jobs missing from Excel: {len(missing_from_excel)}")
        
        if missing_from_excel:
            print(f"   Missing Job IDs (first 10): {list(missing_from_excel)[:10]}")
        
        # Analyze completeness for each job in current batch
        print(f"\n🔬 TREATMENT COMPLETENESS BY JOB (Current Batch)")
        print("-" * 40)
        
        incomplete_jobs = []
        complete_jobs = []
        
        for job_id in current_batch_job_ids:
            if job_id in excel_job_ids:
                # Get treatments for this job from filtered Excel data
                job_treatments = set(current_batch_excel[current_batch_excel['id'] == job_id]['treatment_type'].dropna().astype(str))
                missing_treatments = set(TREATMENT_TYPES) - job_treatments
                
                if missing_treatments:
                    incomplete_jobs.append({
                        'job_id': job_id,
                        'processed_treatments': list(job_treatments),
                        'missing_treatments': list(missing_treatments),
                        'completeness': len(job_treatments) / len(TREATMENT_TYPES) * 100
                    })
                else:
                    complete_jobs.append(job_id)
            else:
                # Job not processed at all
                incomplete_jobs.append({
                    'job_id': job_id,
                    'processed_treatments': [],
                    'missing_treatments': TREATMENT_TYPES,
                    'completeness': 0
                })
        
        print(f"   ✅ Complete Jobs: {len(complete_jobs)}")
        print(f"   ⚠️  Incomplete Jobs: {len(incomplete_jobs)}")
        
        if incomplete_jobs:
            print(f"\n📋 INCOMPLETE JOBS DETAILS")
            print("-" * 40)
            
            # Sort by completeness percentage
            incomplete_jobs.sort(key=lambda x: x['completeness'], reverse=True)
            
            for i, job in enumerate(incomplete_jobs[:10]):  # Show first 10
                print(f"   {i+1:2d}. Job ID: {job['job_id']}")
                print(f"       Completeness: {job['completeness']:.1f}%")
                print(f"       Processed: {job['processed_treatments']}")
                print(f"       Missing: {job['missing_treatments']}")
                print()
            
            if len(incomplete_jobs) > 10:
                print(f"   ... and {len(incomplete_jobs) - 10} more incomplete jobs")
        
        # Summary statistics - ONLY for current batch
        print(f"\n📊 SUMMARY STATISTICS (Current Batch Only)")
        print("-" * 40)
        
        total_expected_operations = len(current_batch_job_ids) * len(TREATMENT_TYPES)
        total_processed_operations = len(current_batch_excel)
        overall_completeness = (total_processed_operations / total_expected_operations) * 100
        
        print(f"   Total Expected Operations: {total_expected_operations}")
        print(f"   Total Processed Operations: {total_processed_operations}")
        print(f"   Overall Completeness: {overall_completeness:.1f}%")
        
        # Treatment-wise analysis - ONLY for current batch
        if 'treatment_type' in current_batch_excel.columns:
            print(f"\n🔬 TREATMENT-WISE ANALYSIS (Current Batch)")
            print("-" * 40)
            
            for treatment in TREATMENT_TYPES:
                treatment_count = len(current_batch_excel[current_batch_excel['treatment_type'] == treatment])
                expected_count = len(current_batch_job_ids)
                treatment_completeness = (treatment_count / expected_count) * 100
                
                print(f"   {treatment}: {treatment_count}/{expected_count} ({treatment_completeness:.1f}%)")
        
        # Recent processing activity - ONLY for current batch
        if 'timestamp' in current_batch_excel.columns:
            print(f"\n⏰ RECENT PROCESSING ACTIVITY (Current Batch)")
            print("-" * 40)
            
            # Convert timestamp to datetime if it's not already
            try:
                current_batch_excel['timestamp'] = pd.to_datetime(current_batch_excel['timestamp'])
                recent_activity = current_batch_excel.sort_values('timestamp', ascending=False).head(5)
                
                print("   Last 5 processed records:")
                for _, row in recent_activity.iterrows():
                    timestamp = row['timestamp'].strftime('%Y-%m-%d %H:%M:%S')
                    job_id = row.get('id', 'N/A')
                    treatment = row.get('treatment_type', 'N/A')
                    print(f"     {timestamp} - Job: {job_id} - Treatment: {treatment}")
            except Exception as e:
                print(f"   Could not parse timestamps: {str(e)}")
    
    else:
        print("   ⚠️  Required columns ('id' or 'treatment_type') not found in Excel file")

print(f"\n{'='*60}")
print("📋 ANALYSIS COMPLETE")
print(f"{'='*60}")

=== EXCEL DATA ANALYSIS AND PROCESSING REPORT ===
✅ Successfully loaded Excel file from: C:\Users\n01780947\OneDrive\Documents\ACCESS_CCSIF_Audit_Study_PDFs\Mapping_data.xlsx
✅ Successfully loaded CSV file: DMC_batch2_resume_matches.csv

📊 EXCEL FILE OVERVIEW
----------------------------------------
📁 File Path: C:\Users\n01780947\OneDrive\Documents\ACCESS_CCSIF_Audit_Study_PDFs\Mapping_data.xlsx
📋 Total Records in Excel: 1402
📋 Total Records in Current Batch: 45
📋 Columns: ['u_id', 'match_id', 'base _resume_file_id', 'treatment_type', 'PDF_google_drive_link', 'resume_filename', 'full_name', 'email', 'phone', 'first_name', 'last_name', 'gender', 'geographic_cluster', 'likely_home_country', 'company', 'job_title', 'job_url_direct', 'job_description', 'resume_final_md', 'match_score', 'Job_location', 'timestamp', 'workflow_mode', 'validation_score', 'validation_json', 'job_id', 'PDF_download_link', 'id']

📈 PROCESSING COMPLETENESS ANALYSIS (Current Batch Only)
---------------------------

C:\Users\n01780947\AppData\Local\Temp\ipykernel_16196\3149735967.py:165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_batch_excel['timestamp'] = pd.to_datetime(current_batch_excel['timestamp'])


### Single File Test (Test Webhook)

In [10]:
# Single File Test
# ===============

print("=== SINGLE FILE TEST ===")
print(f"Test webhook: {TEST_WEBHOOK_URL}")

# Get first file for testing
if 'files_with_pii' in locals() and len(files_with_pii) > 0:
    # test_file = files_with_pii[1]
    test_treatment = 'Type_II' # 'control', 'Type_I', 'Type_II', 'Type_III'    
    # Get specific job ID to test
    test_id = "6914f65a4d906fde395f821a"
    
    # Find the file with matching ID
    test_file = next((f for f in files_with_pii if f['file_row']['_id'] == test_id), files_with_pii[0])
    
    file_row = test_file['file_row']
    file_id = file_row['file_id']
    treatment_pii = test_file['pii_data'][test_treatment]
    
    print(f"\n📄 Testing file: {file_id}")
    print(f"🔬 Treatment: {test_treatment}")
    print(f"👤 PII: {treatment_pii['full_name']} ({treatment_pii['email']})")
    
    # Get template configuration
    template_config = TEMPLATE_ASSIGNMENTS[test_treatment]
    print(f"🎨 Template ID: {template_config['template_id']}")
    print(f"📝 Markdown Template: {template_config['markdown_template']}")
    
    # Create request body
    request_body = create_enhanced_request_body(file_row, test_treatment, treatment_pii)
    request_body['template_id'] = template_config['template_id']
    request_body['markdown_template'] = template_config['markdown_template']
    
    print(f"\n📤 Sending test request...")
    
    try:
        # Send request
        response = requests.post(
            TEST_WEBHOOK_URL,
            json=request_body,
            auth=AUTHORIZATION,
            timeout=120
        )
        
        print(f"📊 Response Status: {response.status_code}")
        
        if response.status_code == 200:
            response_data = response.json()
            
            # Display response data
            print(f"\n✅ Test successful!")
            print(f"📋 Response Data:")
            print(f"   Status: {response_data.get('status', 'N/A')}")
            print(f"   ID: {response_data.get('id', 'N/A')}")
            print(f"   Filename: {response_data.get('file_name', 'N/A')}")
            print(f"   Timestamp: {response_data.get('timestamp', 'N/A')}")
            
            # Google Drive link
            google_drive_link = response_data.get('Google_Drive_Link', '')
            if google_drive_link:
                print(f"   🔗 Google Drive Link: {google_drive_link}")
            else:
                print(f"   ⚠️  Google Drive Link: Not available")
            
            # Full response for debugging
            print(f"\n🔍 Full Response:")
            print(json.dumps(response_data, indent=2))
            
        else:
            print(f"❌ Test failed with status: {response.status_code}")
            print(f"Response: {response.text}")
            
    except Exception as e:
        print(f"❌ Test failed with exception: {str(e)}")
        
else:
    print("❌ No files available for testing")

=== SINGLE FILE TEST ===
Test webhook: https://adhil-is-dummy.app.n8n.cloud/webhook-test/1980a2d5-7906-462b-8d95-ae2ba0579a36

📄 Testing file: FSC Resume 03.pdf
🔬 Treatment: Type_II
👤 PII: Sanjay Khan (khan.alphax04@gmail.com)
🎨 Template ID: 2ee77b23de5fc78e
📝 Markdown Template: 2

📤 Sending test request...
📊 Response Status: 200

✅ Test successful!
📋 Response Data:
   Status: accepted
   ID: {{ $('Webhook').item.json.body.id }}
   Filename: N/A
   Timestamp: N/A
   ⚠️  Google Drive Link: Not available

🔍 Full Response:
{
  "status": "accepted",
  "id": "{{ $('Webhook').item.json.body.id }}",
  "treatment_type": "{{ $('Webhook').item.json.body.treatment_type }}",
  "message": "Processing started - resume will be processed asynchronously"
}
